In [1]:
from repopilot import RepoPilot
from bs4 import BeautifulSoup
from markdownify import markdownify as md
from repopilot.prompts.bug_reproduction import example_br

We will define a template for bug reproduction task. 

In [ ]:
template = "Provide a hypothetical JUnit test case based on the description of the bug report. (without looking at the test folder or avaiable test files) that reproduce the failure behavior of the given bug report as following: \n```{bug_report}```"

Your bug report json file should contain the following fields:

In [ ]:
br = {"issue_id": "CLI-13", "title": "[cli] CommandLine.getOptionValue() behaves contrary to docs", "description": "\n<div class=\"user-content-block\">\n<p>Hi</p>\n<p>If I have:</p>\n<p>final String debugOpt = \"debug\";<br/>\n\tOption debug = OptionBuilder<br/>\n\t    .withArgName(debugOpt)<br/>\n\t    .withDescription(\"turn on debugging\")<br/>\n\t    .withLongOpt(debugOpt)<br/>\n\t    .create('d');</p>\n<p>and then later I do:</p>\n<p>String dbg = commandLine.getOptionValue(debugOpt);</p>\n<p>then dbg will be null. Instead, I have to use getOptionValue('d'). This seems<br/>\ncontrary to the docs (see bottom of<br/>\n<a class=\"external-link\" href=\"http://jakarta.apache.org/commons/cli/usage.html\" rel=\"nofollow noopener\" target=\"_blank\">http://jakarta.apache.org/commons/cli/usage.html</a>), which implies that I should<br/>\nbe able to query the commandLine object using a full string, rather than just<br/>\nthe string's first character.</p>\n<p>Can I suggest that the API of OptionBuilder be made clearer so that it is<br/>\nobvious that you can have long and short option names---perhaps make the<br/>\ncreate() method take no arguments (thus forcing long and short arg names to be<br/>\nset explicitly). (Also, there seems to be some confusion between the terms<br/>\n'argument' and 'option' in the API, but perhaps that is just me).</p>\n<p>Also, I would hop to be able to query commandLine by either a single char or an<br/>\nentire string, as suggested by the docs.</p>\n<p>Thanks,</p>\n<p>Chris</p>\n</div>\n"}

In [ ]:
def make_input(rep_title, rep_content):
    rep_title = BeautifulSoup(rep_title.strip(), 'html.parser').get_text()
    rep_content = md(rep_content.strip())
    bug_report_content = f"# Bug title\n{rep_title}\n## Description\n{rep_content}"
    return bug_report_content

def load_bug_report():
    return make_input(br['title'], br['description'])
    
def query_repopilot_for_gentest(pilot, br):
    print(template.format(bug_report=br))
    output = pilot.query_codebase(template.format(bug_report=br))
    return output 

Please change the api_key to your own openai_api_key.

In [ ]:
api_key = "YOUR_API_KEY"
commit = ""
project = br["issue_id"].split("-")[0]
bug_id = br["issue_id"].split("-")[1]
repo = f"Defects4J/repos/{project}_{bug_id}"

In [ ]:
pilot = RepoPilot(repo, 
        commit=commit, 
        openai_api_key=api_key, 
        language="java", 
        clone_dir="data/repos", 
        examples=example_br)

In [ ]:
output = query_repopilot_for_gentest(pilot, load_bug_report())